## Train GAN on CelebA dataset

In [1]:
import os
import sys

from classes.GAN import GAN
from utils.callbacks import WandbImagesVAE, SaveGeneratorWeights, SaveVAEWeights, WandbVAECallback, WandbImagesGAN, \
    SaveGANWeights
import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback
from imutils import paths
import matplotlib.pyplot as plt

2021-12-14 14:42:02.155017: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-14 14:42:07.079993: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 72677 MB memory:  -> device: 0, name: NVIDIA A100-SXM-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2021-12-14 14:42:07.081910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 72 MB memory:  -> device: 1, name: NVIDIA A100-SXM-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2021-12-14 14:42:07.084272: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GP

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

### Model definition
And initial configuration

In [29]:
wandb.login()

latent_dim=512
input_shape=(128,128,3)

encoder_architecture=[(0,32),(0,64),(1,128),(1,128),(1,256)]
decoder_architecture=[(0,256),(0,128),(1,128),(1,64),(1,32)]

g=GAN(input_shape,
      latent_dim=latent_dim,
      encoder_architecture=encoder_architecture,
      decoder_architecture=decoder_architecture)


config={"dataset":"celebA", "type":"GAN","encoder_architecture":encoder_architecture,"decoder_architecture":decoder_architecture}
config.update(g.get_dict())


images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## Generator

In [12]:
g.generator.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 latent_input (InputLayer)   [(None, 512)]             0         
                                                                 
 dense_3 (Dense)             (None, 1024)              525312    
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 64)          0         
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 4, 4, 64)          0         
                                                                 
 conv_transpose_res_block_5   (None, 8, 8, 256)        147712    
 (ConvTransposeResBlock)                                         
                                                                 
 conv_transpose_res_block_6   (None, 16, 16, 128)      295040    
 (ConvTransposeResBlock)                                   

## Discriminator

In [13]:
g.discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 128, 128, 3)]     0         
                                                                 
 conv_res_block_5 (ConvResBl  (None, 64, 64, 32)       896       
 ock)                                                            
                                                                 
 conv_res_block_6 (ConvResBl  (None, 32, 32, 64)       18496     
 ock)                                                            
                                                                 
 conv_res_block_7 (ConvResBl  (None, 16, 16, 128)      386560    
 ock)                                                            
                                                                 
 conv_res_block_8 (ConvResBl  (None, 8, 8, 128)        460288    
 ock)                                                

In [14]:
#other important definitions

EPOCHS=250
BS=512
INIT_LR=5e-5

config["epochs"]=EPOCHS
config["BS"]=BS
config["init_lr"]=INIT_LR

config["nota"]="DGX"

wandb.init(project="TorVergataExperiment-Generative",config=config)

## Dataloaders

In [15]:
def load_images(imagePath):
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128)) / 255.0

    # eventually load other information like attributes here

    # return the image and the extra info

    return image


print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")


train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)
train_dataset = (train_dataset
    .shuffle(1024)
    .map(load_images)
    .cache()
    .repeat()
    .batch(BS)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices(val_imgs)
val_dataset = (val_dataset
    .shuffle(1024)
    .map(load_images)
    .cache()
    .repeat()
    .batch(BS)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices(test_imgs)
test_dataset = (test_dataset
    .shuffle(1024)
    .map(load_images)
    .cache()
    .batch(BS)
)


[INFO] loading image paths...
[TRAINING]	 138545
[VALIDATION]	 17318
[TEST]		 17319


## Compile
And set callbacks

In [31]:
os.makedirs("models/gan",exist_ok=True)
model_check=SaveGANWeights(filepath="models/gan")

g.compile()


try:                                  #workaround to use Wandbcallback at first attempt
    wb=WandbCallback()
except:
    wb=WandbCallback()
    
    
callbacks=[
    WandbImagesGAN(target_shape=(128,128,3)),
    wb,
    model_check,
]


## Train

In [ ]:
#just to build the model
x=np.zeros((BS,*input_shape))
g(x)

g.fit(train_dataset,validation_data=test_dataset,steps_per_epoch=ts,validation_steps=vs,epochs=EPOCHS,callbacks=callbacks)


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Epoch 1/250
270/270 [==============================] - ETA: 0s - d_loss: -22013.5248 - g_loss: 15396427.7773 - d_acc: 0.1121

2021-12-14 14:52:35.032592: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


270/270 [==============================] - 144s 522ms/step - d_loss: -21917.7915 - g_loss: 15339709.9256 - d_acc: 0.1118 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/250
270/270 [==============================] - ETA: 0s - d_loss: 4241.5482 - g_loss: 11933.4230 - d_acc: 1.8454e-05

2021-12-14 14:54:57.222151: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


270/270 [==============================] - 142s 526ms/step - d_loss: 4241.7092 - g_loss: 12075.1970 - d_acc: 1.8426e-05 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/250
270/270 [==============================] - ETA: 0s - d_loss: 2231.5099 - g_loss: 13465.0404 - d_acc: 0.0000e+00

...

270/270 [==============================] - 132s 488ms/step - d_loss: 3.3602 - g_loss: 582.9295 - d_acc: 1.6280e-04 - val_loss: 0.0000e+00 - val_accuracy: 3.8471e-04
Epoch 29/250
270/270 [==============================] - ETA: 0s - d_loss: 37.3338 - g_loss: 538.1000 - d_acc: 1.7926e-04

2021-12-14 15:54:08.634127: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


270/270 [==============================] - 132s 488ms/step - d_loss: 37.5078 - g_loss: 538.2860 - d_acc: 1.7933e-04 - val_loss: 0.0000e+00 - val_accuracy: 1.7756e-04
Epoch 30/250
270/270 [==============================] - ETA: 0s - d_loss: 9.3315 - g_loss: 190.2759 - d_acc: 1.9985e-04

2021-12-14 15:56:20.114130: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


270/270 [==============================] - 131s 487ms/step - d_loss: 9.3112 - g_loss: 189.9462 - d_acc: 2.0011e-04 - val_loss: 0.0000e+00 - val_accuracy: 2.9593e-04
Epoch 31/250
268/270 [============================>.] - ETA: 0s - d_loss: 1.7585 - g_loss: 149.2164 - d_acc: 3.6462e-04